# Telugu News Article Classification Using Natural Language Processing

**Problem Statement:** In this notebook, I tried to classify news paper articles using Natural Language Processing in TensorFlow

* **Data:** The data is taken from Kaggle datasets. You can find the data [here]('https://www.kaggle.com/sudalairajkumar/telugu-nlp?select=telugu_news').

* **Evaluation:** We used accuracy to evaluate the model

I've tried various models for text classification like
* Naive Bayes model
* FeedForward Neural Network
* LSTM
* GRU
* Bidirectional GRU

and found that GRU model performs better with accuracy of ***93.69%*** accuracy.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

--2022-02-19 13:30:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-02-19 13:30:52 (83.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [68]:
# Function to convert pred probs into labels
def make_preds(model):
    """
    This function make predictions on the validation data and returns the prediction labels
    """
    pred_probs = model.predict(test_data)
    preds = tf.argmax(pred_probs, axis=1)
    
    return preds

In [69]:
# Make a function some predictions on test data and visualize them
def visualize_random_preds(model):
    """
    Make predictions with our model and visualize them with their labels
    r
    Args:
        model: model you want to predict the labels with
         
    """
    
    
    # Selecting a random sentence from our validation dataset
    random_number = random.randint(0, len(test_sentences))
    y_preds = make_preds(model)
    print(f"Text: \n{test_sentences[random_number]}\n")
    print(f"Predicted label: {class_names[y_preds[random_number]]}\n")
    print(f"Original label: {test_labels[random_number]}\n" )



In [6]:
train = pd.read_csv("https://raw.githubusercontent.com/amulyaprasanth/telugu-news-nlp/main/train_telugu_news.csv?token=GHSAT0AAAAAABRUD26EMMTNW7DJCP2RMRSMYQ2HQIA")
test = pd.read_csv("https://raw.githubusercontent.com/amulyaprasanth/telugu-news-nlp/main/test_telugu_news.csv?token=GHSAT0AAAAAABRUD26FEIFCFECPDR4QRQWIYQ2HSFQ")

In [8]:
train.head()

,SNo,date,heading,body,topic
0,414,11-05-2017 00:39:13,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business
1,2668,12-06-2017 00:40:32,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business
2,19923,19-01-2017 19:51:31,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
3,15762,23-11-2017 17:29:04,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation
4,8624,03-04-2017 15:48:23,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment


In [9]:
test.head()

,SNo,date,heading,body,topic
0,7771,19-05-2017 13:44:10,కేశవ,హీరోగా తెలుగు సినిమాల్లోకి ఎంట్రీ ఇచ్చిన నిఖిల...,entertainment
1,9591,01-08-2017 13:58:24,డబ్బుల కోసం ప్రాణాల మీదకు తెచ్చుకుంటారా.. హీరో...,సినిమాల్లో యాక్షన్ స్టంట్లు చేసేటప్పుడు ఎక్కు...,entertainment
2,12622,05-04-2017 00:31:58,దక్షిణాదిన బీజేపీ పప్పులుడకవ్‌,దక్షిణ భారతాన్ని ఆక్రమించేందుకు బీజేపీ పంచెలు ...,nation
3,16401,30-08-2017 03:34:37,పీఎస్‌ఎల్వీ-సీ39కి గ్రీన్‌సిగ్నల్‌,నేటి మధ్యాహ్నం 1.59కి కౌంట్‌డౌన్‌ షురూశ్రీహరి...,nation
4,10422,10-04-2017 09:00:48,ఏయ్ జయప్రకాష్ అనే కేక వినిపించింది.. వెనక్కి త...,‘ఏమి రా... అబ్బి... యాడికి పోయినావు’ అంటూ రాయ...,entertainment


In [10]:
# How many classes are there
train["topic"].value_counts()

nation           6628
entertainment    5145
business         2572
sports           1908
editorial        1059
Name: topic, dtype: int64

In [11]:
# Check the data types and shapes of our datasets
train.shape, test.shape

((17312, 5), (4329, 5))

In [12]:
# Before preprocessing our data let's extract the features we need to work with 
features = train["body"].to_numpy()
labels = train["topic"].to_numpy()

test_sentences = test["body"].to_numpy()
test_labels = test["topic"].to_numpy()

In [13]:
# Split our data into train and validation_sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Check the length of valid and train datasets
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(13849, 13849, 3463, 3463)

In [14]:
# Convert our labels into numbers
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
train_labels_encoded = enc.fit_transform(train_labels)
val_labels_encoded = enc.transform(val_labels)
test_labels_encoded = enc.transform(test_labels)

In [15]:
class_names = np.unique(train_labels)
class_names

array(['business', 'editorial', 'entertainment', 'nation', 'sports'],
      dtype=object)

## Becoming one with the data (Inspecting the data)

In [16]:
len(features)

17312

In [17]:
# visuolise random train data and its labels
import random 
random_sentence = random.choice(features)
print(random_sentence)

విడాకులు తీసుకున్న హీరోయిన్ కు అవకాశాలు ఏమోస్తాయో అనుకున్న వారంతా ఆమె స్కిన్ షో చూసి తమ అభిప్రాయం మార్చుకుంటున్నారట. రీసెంట్‌గా అందాల ఆరబోతలో మరో మెట్టు పైకి ఎక్కిందట ఆ ముద్దుగుమ్మ. అందాల భామలు హాట్‌గా కనిపిస్తుంటారు. అందులో ఆశ్చర్యపోవాల్సి పని కూడా లేదు. కానీ కొందరు హీరోయిన్లు మోతాదుకు మించి అందాలు ఆరబోస్తే అది స్పెషల్ అట్రాక్షన్‌గా మారుతుంది. రీసెంట్‌గా ఓ మ్యాగజైన్‌కు అమలా పాల్ రెచ్చిపోయి ఫోజులు ఇవ్వడం కూడా చాలామంది దృష్టిని ఆకర్షిస్తోంది. పెళ్లి తరువాత కూడా హీరోయిన్‌గా నటించాలని భావించిన అమలా పాల్... అందుకు తన పెళ్లి బంధం అడ్డురావడంతో భర్త నుంచి విడిపోవడానికి కూడా వెనకాడలేదు. పెళ్ళి పెటాకులు అయిన తర్వాత ఫ్రీ బర్డ్ అయిన ఈ అందాల భామ... ఆ మధ్య వచ్చిన కన్నడ మూవీలో ఫుల్లుగా గ్లామర్ ఒలకబోసిందనే ప్రచారం జరిగింది. సుదీప్ హీరోగా తెరకెక్కిన ఈ సినిమాలో అమలను అందాల ప్రదర్శన కోసమే తీసుకున్నారనే గుసగుసలు కూడా వినిపించాయి. అయితే తనకు కావాల్సింది కూడా అలాంటి అవకాశాలే కావడంతో ఇందుకోసం మరోసారి ఫోటోషూట్‌లో రెచ్చిపోయి స్కిన్ షో చేసింది ఈ మలయాళ భామ. రీసెంట్‌గా జేఎఫ్‌డబ్ల్యూ మ్యాగజైన్ కోసం ఫోటో షూట్‌లో ప

### Creating a baseline model

This is a model we've to beat


In [18]:
# Let's train a naive bayes model using tf-idf-vectorizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

baseline_model = Pipeline([
    ("tfidf_vectorizer", TfidfVectorizer()),
    ("naive_bayes", MultinomialNB())
])

baseline_model.fit(train_sentences, train_labels_encoded)

Pipeline(steps=[('tfidf_vectorizer', TfidfVectorizer()),
                ('naive_bayes', MultinomialNB())])

In [19]:
# Evaluate the base model 
baseline_model.score(val_sentences, val_labels_encoded)

0.7403984984117817

In [20]:
# Making predictions with our model
baseline_preds = baseline_model.predict(val_sentences)
baseline_preds[:10], len(baseline_preds)

(array([2, 3, 3, 3, 2, 3, 3, 3, 3, 3]), 3463)

In [21]:
# Get basemodel results
baseline_results = calculate_results(val_labels_encoded, baseline_preds)
baseline_results

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 74.03984984117817,
 'f1': 0.7024314343853166,
 'precision': 0.766110761383885,
 'recall': 0.7403984984117817}

In [22]:
# Visualize predictions
visualize_random_preds(baseline_model, baseline_preds)

Text: 
హైదరాబాద్: అమర్‌నాథ్ యాత్రికులపై ఉగ్రవాదులు జరిపిన పాశవిక దాడిని ఏఐఎంఐఎం నేత అసదుద్దీన్ ఒవైసీ ఖండించారు. పాకిస్థాన్‌పై నిప్పులు చెరిగారు. ఈ దాడి వెనుక పాక్ ఉగ్రవాద గ్రూపులు ఉన్నాయన్నారు. సోమవారం రాత్రి భక్తులు అమర్‌నాథ్ యాత్ర ముగించుకుని బస్సులో వెళ్తుండగా అనంతనాగ్ జిల్లాల్లో ఉగ్రవాదులు కాల్పులతో విరుచుకుపడ్డారు. ఈ ఘటనలో ఆరుగురు మహిళలతో సహా ఏడుగురు ప్రాణాలు కోల్పోయారు. మరో 32 మందికి పైగా గాయపడ్డారు. ఈ ఘటనను ఖండించిన ఒవైసీ...లష్కర్, ఐఎస్ఐల దుష్టపన్నాగాలను సాగనీయమని అన్నారు. దేశం ఐక్యంగా ఉందన్నారు. ఉగ్రవాదుల దాడి అత్యంత హేయమైనదని తూర్పారబట్టారు. కాగా, అమర్‌నాథ్ యాత్రపై 2001లోనూ ఉగ్రవాదాలు దాడి జరిపారు. అమర్‌నాథ్‌ గుహకు సమీపంలోని శేష్‌నాగ్ ఏరియాలో ఉగ్రవాదులు జరిపిన దాడిలో 13 మంది యాత్రికులు మృతిచెందారు. దీనికి ముందు 2000, ఆగస్టు 1న పహల్‌గామ్ ఏరియాలో అమర్‌నాథ్ యాత్రికులపై ఉగ్రవాదాలు జరిపిన కాల్పుల్లో పోర్టర్లతో సహా 30 మంది ప్రాణాలు కోల్పోయారు.

Predicted label: nation

Original label: nation



## Preprocess the data

Before going to Neural Networks we need to turn our data into numerical format in which our model can understand

* TextVectorization  (Converting our text into numbers)
* Embedding layer ( Creating embeddings from the text vectors)

In [23]:
from tensorflow.keras.layers import TextVectorization, Embedding

In [24]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

172

In [25]:
# Instantiating our text vectorization layer with required parameters
max_vocab_length = 10000 # Max number of words in our vocabulary
max_length = 173 # Output sequence length
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_sequence_length=max_length)

In [26]:
# Fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [27]:
# Test out the text vectorization layer
# sample_sentence = "ఇండియ‌న్ వారెన్ బ‌ఫెట్ రాకేష్ ఝున్‌ఝున్ వాలా ఐటీ ఉద్యోగాల‌పై ఆస‌క్తిక‌ర విష‌యాల్ని వెల్ల‌డించారు."
# text_vectorizer([sample_sentence]) # The text vectorization layer is working fine

In [28]:
# Creating embedding layer
embedding_1 = Embedding(input_dim=max_vocab_length,
                       output_dim=128,
                       embeddings_initializer='uniform',
                       input_length=max_length,
                       name="embedding_1")

In [29]:
# Test out the embedding layer
# sample_sentence = "ఇండియ‌న్ వారెన్ బ‌ఫెట్ రాకేష్ ఝున్‌ఝున్ వాలా ఐటీ ఉద్యోగాల‌పై ఆస‌క్తిక‌ర విష‌యాల్ని వెల్ల‌డించారు."
# sample_embedding = embedding_1(text_vectorizer([sample_sentence]))
# sample_embedding

In [30]:
# turn our data into tensorflow datasets
train_sentences_dataset = tf.data.Dataset.from_tensor_slices(train_sentences)
train_labels_encoded_dataset = tf.data.Dataset.from_tensor_slices(train_labels_encoded)
train_data = tf.data.Dataset.zip((train_sentences_dataset, train_labels_encoded_dataset)).batch(32).prefetch(tf.data.AUTOTUNE)

val_sentences_dataset = tf.data.Dataset.from_tensor_slices(val_sentences)
val_labels_encoded_dataset = tf.data.Dataset.from_tensor_slices(val_labels_encoded)
val_data = tf.data.Dataset.zip((val_sentences_dataset, val_labels_encoded_dataset)).batch(32).prefetch(tf.data.AUTOTUNE)

test_sentences_dataset = tf.data.Dataset.from_tensor_slices(test_sentences)
test_labels_encoded_dataset = tf.data.Dataset.from_tensor_slices(test_labels_encoded)
test_data = tf.data.Dataset.zip((test_sentences_dataset, test_labels_encoded_dataset)).batch(32).prefetch(tf.data.AUTOTUNE)

train_data, val_data, test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

## Let's build some models
We've converted our text into numbers.
Now let's build some models
* Dense model
* LSTM
* GRU
* Bidirectional LSTM
* 1D Conv network
* Transfer Learning

### Dense model

In [31]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, GRU, Conv1D, Bidirectional, Input, GlobalAveragePooling1D, GlobalMaxPooling1D

In [32]:
# Create dense model with Functinal API
inputs = Input(shape=(1,), dtype="string", name="input_layer")
x = text_vectorizer(inputs)
x = embedding_1(x)
x = GlobalAveragePooling1D(name='global_pooling_layer')(x)
outputs = Dense(5, activation="softmax", name="output_layer")(x)

model_1 = Model(inputs, outputs, name="dense_model")

In [33]:
# Check the summary of the model
model_1.summary()

Model: "dense_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 173)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 173, 128)          1280000   
                                                                 
 global_pooling_layer (Globa  (None, 128)              0         
 lAveragePooling1D)                                              
                                                                 
 output_layer (Dense)        (None, 5)                 645       
                                                                 
Total params: 1,280,645
Trainable params: 1,280,645
Non

In [34]:
# Compile the model
model_1.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

In [35]:
# Create callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,
                                                 restore_best_weights=True,
                                                 monitor="val_accuracy")

In [36]:
# Fit the model
dense_model_history = model_1.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch=len(train_data),
                                 validation_data=val_data,
                                 validation_steps=len(val_data),
                                 callbacks=[create_tensorboard_callback(SAVE_DIR, model_1.name)])

Saving TensorBoard log files to: model_logs/dense_model/20220219-133216
Epoch 1/10
433/433 [==============================] - 8s 12ms/step - loss: 1.0697 - accuracy: 0.6338 - val_loss: 0.6355 - val_accuracy: 0.8031
Epoch 2/10
433/433 [==============================] - 5s 12ms/step - loss: 0.4397 - accuracy: 0.8841 - val_loss: 0.3568 - val_accuracy: 0.8929
Epoch 3/10
433/433 [==============================] - 8s 18ms/step - loss: 0.2640 - accuracy: 0.9307 - val_loss: 0.2702 - val_accuracy: 0.9116
Epoch 4/10
433/433 [==============================] - 5s 12ms/step - loss: 0.1879 - accuracy: 0.9508 - val_loss: 0.2282 - val_accuracy: 0.9261
Epoch 5/10
433/433 [==============================] - 6s 14ms/step - loss: 0.1424 - accuracy: 0.9636 - val_loss: 0.2042 - val_accuracy: 0.9339
Epoch 6/10
433/433 [==============================] - 5s 12ms/step - loss: 0.1116 - accuracy: 0.9737 - val_loss: 0.1898 - val_accuracy: 0.9396
Epoch 7/10
433/433 [==============================] - 7s 16ms/step - l

In [63]:
# calculate results
dense_results = model_1.evaluate(test_data)
dense_results

136/136 [==============================] - 1s 10ms/step - loss: 0.2770 - accuracy: 0.9023


[0.27703845500946045, 0.9022868871688843]

In [39]:
# Visualise predictons
visualize_random_preds(model_1, model_1_preds)

Text: 
హైదరాబాద్‌ (ఆంధ్రజ్యోతి): యుఎస్‌ఐఎస్‌సి ఫోరం ప్రతినిధులు తెలంగాణ రాష్ట్ర ఐటి, పరిశ్రమల శాఖ  మంత్రి కె తారక రామారావుతో సమావేశమయ్యారు. నవంబరులో జరగనున్న ప్రపంచ ఔత్సాహిక పారిశ్రామికవేత్తల సదస్సుపై ఈ భేటీలో చర్చించారు.  అమెరికా-భారత్‌ సంయుక్తంగా తొలిసారిగా ఈ సదస్సును నిర్వహించనున్నాయి.  ఈ సదస్సుకి ప్రధాన మంత్రి నరేంద్ర మోదీతో పాటు అమెరికా అధ్యక్షుడు డొనాల్డ్‌ ట్రంప్‌ కుమార్తె  ఇవాంకా హాజరుకానున్నారు. ఈ నేపఽథ్యంలో సదస్సు నిర్వహణపై ఫోరం అధ్యక్షుడు ముకే్‌షతో పాటు పలువురు ప్రతినిధులు మంత్రితో సమావేశమయ్యారు.

Predicted label: business

Original label: business



### LSTM model

In [45]:
# Recreate another embedding layer
embedding_2 = Embedding(input_dim=max_vocab_length,
                       output_dim=128,
                        input_length=max_length,
                       name="embedding_2")

# Create LSTM model with functional API
inputs = Input(shape=(1,), dtype="string", name="input_layer")
x = text_vectorizer(inputs)
x = embedding_2(x)
# x = LSTM(64, return_sequences=True)(x)
x = LSTM(64)(x)
outputs = Dense(5, activation="softmax", name="output_layer")(x)

model_2 = Model(inputs, outputs, name="lstm_model")

# Compile the model
model_2.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])


# Fit the model
lstm_model_history = model_2.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch=len(train_data),
                                 validation_data=val_data,
                                 validation_steps=len(val_data),
                                 callbacks=[create_tensorboard_callback(SAVE_DIR, model_2.name)])

Saving TensorBoard log files to: model_logs/lstm_model/20220219-134350
Epoch 1/10
433/433 [==============================] - 23s 48ms/step - loss: 1.3655 - accuracy: 0.4058 - val_loss: 1.3029 - val_accuracy: 0.4369
Epoch 2/10
433/433 [==============================] - 19s 45ms/step - loss: 1.1429 - accuracy: 0.5174 - val_loss: 1.5652 - val_accuracy: 0.3921
Epoch 3/10
433/433 [==============================] - 20s 45ms/step - loss: 1.1706 - accuracy: 0.5033 - val_loss: 1.0904 - val_accuracy: 0.5316
Epoch 4/10
433/433 [==============================] - 19s 45ms/step - loss: 1.2285 - accuracy: 0.4755 - val_loss: 1.1685 - val_accuracy: 0.4895
Epoch 5/10
433/433 [==============================] - 19s 44ms/step - loss: 0.8922 - accuracy: 0.6482 - val_loss: 0.9721 - val_accuracy: 0.5585
Epoch 6/10
433/433 [==============================] - 20s 45ms/step - loss: 0.8256 - accuracy: 0.6514 - val_loss: 0.7751 - val_accuracy: 0.6904
Epoch 7/10
433/433 [==============================] - 19s 44ms/st

In [46]:
# Calculate results
lstm_results = calculate_results(val_labels_encoded, make_preds(model_2))
lstm_results

{'accuracy': 49.98556165174704,
 'f1': 0.445103750336797,
 'precision': 0.6565135544381686,
 'recall': 0.4998556165174704}

In [64]:
# Evaluate on test data
model_2.evaluate(test_data)

136/136 [==============================] - 3s 21ms/step - loss: 1.2455 - accuracy: 0.5040


[1.2455335855484009, 0.5040425062179565]

### GRU model

In [47]:
# Recreate another embedding layer
embedding_3 = Embedding(input_dim=max_vocab_length,
                       output_dim=128,
                        input_length=max_length,
                       name="embedding_3")

# Create LSTM model with functional API
inputs = Input(shape=(1,), dtype="string", name="input_layer")
x = text_vectorizer(inputs)
x = embedding_3(x)
x = GRU(64)(x)
outputs = Dense(5, activation="softmax", name="output_layer")(x)

model_3 = Model(inputs, outputs, name="gru_model")

# Compile the model
model_3.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])


# Fit the model
gru_model_history = model_3.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch=len(train_data),
                                 validation_data=val_data,
                                 validation_steps=len(val_data),
                                 callbacks=[create_tensorboard_callback(SAVE_DIR, model_3.name)])

Saving TensorBoard log files to: model_logs/gru_model/20220219-134736
Epoch 1/10
433/433 [==============================] - 22s 46ms/step - loss: 1.3840 - accuracy: 0.3988 - val_loss: 1.2557 - val_accuracy: 0.4531
Epoch 2/10
433/433 [==============================] - 19s 45ms/step - loss: 0.4823 - accuracy: 0.8358 - val_loss: 0.2314 - val_accuracy: 0.9327
Epoch 3/10
433/433 [==============================] - 19s 45ms/step - loss: 0.1192 - accuracy: 0.9655 - val_loss: 0.2130 - val_accuracy: 0.9353
Epoch 4/10
433/433 [==============================] - 19s 44ms/step - loss: 0.0527 - accuracy: 0.9853 - val_loss: 0.2512 - val_accuracy: 0.9307
Epoch 5/10
433/433 [==============================] - 19s 45ms/step - loss: 0.0273 - accuracy: 0.9934 - val_loss: 0.2572 - val_accuracy: 0.9373
Epoch 6/10
433/433 [==============================] - 20s 45ms/step - loss: 0.0153 - accuracy: 0.9960 - val_loss: 0.2853 - val_accuracy: 0.9353
Epoch 7/10
433/433 [==============================] - 20s 45ms/ste

In [48]:
# Calculate gru results
gru_results = calculate_results(val_labels_encoded, make_preds(model_3))
gru_results

{'accuracy': 92.95408605255558,
 'f1': 0.9284437658857343,
 'precision': 0.93052369447302,
 'recall': 0.9295408605255558}

In [65]:
model_3.evaluate(test_data)

136/136 [==============================] - 3s 22ms/step - loss: 0.3208 - accuracy: 0.9369


[0.3208496868610382, 0.9369369149208069]

### Bidirectional model


In [49]:
# Recreate another embedding layer
embedding_4 = Embedding(input_dim=max_vocab_length,
                       output_dim=128,
                        input_length=max_length,
                       name="embedding_4")

# Create LSTM model with functional API
inputs = Input(shape=(1,), dtype="string", name="input_layer")
x = text_vectorizer(inputs)
x = embedding_4(x)
x = Bidirectional(GRU(64))(x)
outputs = Dense(5, activation="softmax", name="output_layer")(x)

model_4 = Model(inputs, outputs, name="bidirectional_model")

# Compile the model
model_4.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])


# Fit the model
bidirectional_model_history = model_4.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch=len(train_data),
                                 validation_data=val_data,
                                 validation_steps=len(val_data),
                                 callbacks=[create_tensorboard_callback(SAVE_DIR, model_3.name)])

Saving TensorBoard log files to: model_logs/gru_model/20220219-135508
Epoch 1/10
433/433 [==============================] - 43s 84ms/step - loss: 0.5704 - accuracy: 0.7873 - val_loss: 0.2645 - val_accuracy: 0.9062
Epoch 2/10
433/433 [==============================] - 35s 82ms/step - loss: 0.1426 - accuracy: 0.9546 - val_loss: 0.2455 - val_accuracy: 0.9252
Epoch 3/10
433/433 [==============================] - 35s 81ms/step - loss: 0.0596 - accuracy: 0.9831 - val_loss: 0.2880 - val_accuracy: 0.9249
Epoch 4/10
433/433 [==============================] - 35s 82ms/step - loss: 0.0292 - accuracy: 0.9926 - val_loss: 0.2786 - val_accuracy: 0.9275
Epoch 5/10
433/433 [==============================] - 35s 82ms/step - loss: 0.0137 - accuracy: 0.9962 - val_loss: 0.3253 - val_accuracy: 0.9252
Epoch 6/10
433/433 [==============================] - 35s 81ms/step - loss: 0.0095 - accuracy: 0.9973 - val_loss: 0.3956 - val_accuracy: 0.9180
Epoch 7/10
433/433 [==============================] - 35s 82ms/ste

In [50]:
# Calculate bidirectional results
bidirectional_results = calculate_results(val_labels_encoded, make_preds(model_4))
bidirectional_results

{'accuracy': 92.26104533641352,
 'f1': 0.9206923918858719,
 'precision': 0.9215861155976968,
 'recall': 0.9226104533641352}

In [66]:
model_4.evaluate(test_data)

136/136 [==============================] - 5s 34ms/step - loss: 0.4021 - accuracy: 0.9282


[0.4020584523677826, 0.9281589388847351]

In [70]:
# Visualise model predictions
visualize_random_preds(model_3)

Text: 
ఆర్ట్‌ ఆఫ్‌ లివింగ్‌ నుంచి అనుమానాస్పద యోగా!ప్రజ్ఞా యోగ పేరిట ‘అతీంద్రియ’ ప్రచారంగంతలు కట్టినా పిల్లలు చదువుతారట!గోడకు అవతల ఏముందో చెబుతారట!భవిష్యత్తునూ ఊహించే శక్తి వస్తుంది!నాలుగు గంటల శిక్షణకు 10 వేల ఫీజుఇదంతా ‘ట్రిక్‌’ అంటున్న మెజీషియన్లుయోగా... మనసుకు ప్రశాంతతను ఇస్తుంది. శరీరానికి ఆరోగ్యాన్ని కూడా ఇస్తుంది. ఇది అందరికీ తెలిసిన విషయం. అందుకే ప్రపంచమంతా నేడు యోగ ముద్ర వేస్తోంది. మరి... ఇదే యోగాలోని ఒక ప్రక్రియ వల్ల ‘భవిష్యత్తును ఊహించగలిగే’ శక్తి వస్తుందా? పరీక్షల్లో రాబోయే ప్రశ్నలు ముందే తెలిసిపోతాయా? కళ్లకు గంతలు కట్టినా, కనపడకున్నా చదివేస్తారా? నాస్తికుల సంగతి పక్కనపెడితే... హేతుబద్ధంగా ఆలోచించే వారెవరూ ఈ ప్రశ్నలకు ‘ఔను’ అని సమాధానం చెప్పరు. ‘ప్రజ్ఞా యోగ’ అనే ప్రక్రియతో పిల్లలకు ఇలాంటి అద్భుత శక్తులు వస్తాయట! వాళ్లూ వీళ్లూ... ఊరూపేరు లేని వాళ్లు కాదు! పద్మ విభూషణ్‌, ప్రపంచ ప్రఖ్యాతి గాంచిన ఆధ్యాత్మిక వేత్త శ్రీశ్రీ రవిశంకర్‌  గురూజీ ఆధ్వర్యంలో నడుస్తున్న ఆర్ట్‌ ఆఫ్‌ లివింగ్‌ స్వయంగా చెబుతున్న మాట ఇది! ‘ప్రజ్ఞా యోగ’ పేరిట ఇప్పుడు భారీ స్థాయి ప్రచారం జరుగుతోంది. గురూజీ నిజం